In [83]:
%load_ext autoreload
%autoreload 2
from gui import SpikeImportGUI

spike_gui = SpikeImportGUI()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [85]:
print(spike_gui.time_channel)
print(spike_gui.stimulus_channels)

Time
{'regular_electrical': '801 DigMark', 'extra_electrical': '32 DigMark'}


In [60]:
import tkinter as tk
import pygubu

builder = pygubu.Builder()
builder.add_from_file('./gui/file_import.ui')

main_window = builder.get_object('toplevel_window')
file_spike = builder.get_object('file_spike')
btn_snoop = builder.get_object('btn_snoop')
btn_loadfile = builder.get_object('btn_loadfile')

# list of channels in the current file
channels = []

load_mech = tk.BooleanVar()
load_ex_el = tk.BooleanVar()

def switch_sel(sel, enable):
    if enable == True:
        sel['state'] = "normal"
    else:
        sel['state'] = "disabled"

def get_channel_names(filepath):
    import pandas as pd
    
    # read the first csv line using pandas to get the column names
    file_head = pd.read_csv(filepath_or_buffer = filepath, nrows = 1, header = 0)
    channels = file_head.columns.values
    channels = [channel.replace("'", "\"") for channel in channels]

    return channels

def snoop_spikefile():
    filepath = file_spike['path']
    
    # get the channel names
    channels = get_channel_names(filepath)
    
    # activate all of the combo boxes
    comboboxes = ["sel_time", "sel_signal", "sel_reg_el"]
    for box in comboboxes:
        box_obj = builder.get_object(box)
        box_obj['state'] = "normal"
        box_obj['values'] = channels
        
    builder.get_object("sel_mech")['values'] = channels
    builder.get_object("sel_ex_el")['values'] = channels
        
    # initialize checkboxes
    chk_mech = builder.get_object('chk_mech')
    chk_mech['state'] = "normal"
    
    chk_mech['variable'] = load_mech
    chk_mech['command'] = lambda: switch_sel(builder.get_object('sel_mech'), load_mech.get())
    
    chk_ex_el = builder.get_object('chk_ex_el')
    chk_ex_el['state'] = "normal"
    chk_ex_el['variable'] = load_ex_el
    chk_ex_el['command'] = lambda: switch_sel(builder.get_object('sel_ex_el'), load_ex_el.get())
    
    # initialize the AP channel list box
    lst_ap = builder.get_object('lst_ap')
    lst_ap['state'] = "normal"
    for ch in channels:
        lst_ap.insert(tk.END, ch)

x = None
        
def load_spikefile():
    from importers import SpikeImporter
    
    filepath = file_spike['path']
    
    # get the channel names
    channels = get_channel_names(filepath)
    
    # get names of the chosen time and signal channels
    time_channel = builder.get_object('sel_time').get()
    signal_channel = builder.get_object('sel_signal').get()
    
    # get the names of the stimulus channels (if desired)
    stimulus_channels = dict()
    stimulus_channels["regular_electrical"] = builder.get_object('sel_reg_el').get()
    
    if load_mech.get() == True:
        stimulus_channels["force"] = builder.get_object('sel_mech').get()
        
    if load_ex_el.get() == True:
        stimulus_channels["extra_electrical"] = builder.get_object('sel_ex_el').get()
        
    # get the AP channels
    lst_ap = builder.get_object('lst_ap')
    ap_channels = [channels[idx] for idx in lst_ap.curselection()]
        
    print(stimulus_channels)
    print(ap_channels)
        
    x = SpikeImporter(filepath = filepath, time_channel = time_channel, signal_channel = signal_channel), stimulus_channels, ap_channels
        

    
btn_snoop['command'] = snoop_spikefile
btn_loadfile['command'] = load_spikefile
main_window.mainloop()

print(x)

(3,)
{'regular_electrical': '801 DigMark', 'extra_electrical': '32 DigMark'}
['3 nw-1']
(1, 2, 3)
{'regular_electrical': '801 DigMark', 'extra_electrical': '32 DigMark'}
['801 DigMark', '32 DigMark', '3 nw-1']
None


In [59]:
print(channels)

[]


In [55]:
%load_ext autoreload
%autoreload 2

from importers import SpikeImporter

data_folder = "../data/"
file_name = "11_10_27U2a_2000s_2300s_20000Hz_Force_AllFibres"
file_ending = ".csv"

time_channel = "Time"
signal_channel = "1 Signal"
stimulus_channel = "32 DigMark"
ap_marker_channels = ["801 nw-1", "12 nw-1", "14 nw-1"]
force_channel = "3 Force"
# extra_stimuli_channel = "801 DigMark"

# load the animal file
importer = SpikeImporter(filepath = data_folder + file_name + file_ending, time_channel = time_channel, signal_channel = signal_channel)

print("Data loaded.")

# define the max. time so that two sequential waveform-values are belonging to the same AP
# the time is in seconds, so 0.005 equals 5ms
max_gap_time = 0.005
force_thresh = 0.5

# retrieve the events from the importer 
el_stimuli = importer.get_electrical_stimuli(regular_stimulus_channel = stimulus_channel)
ex_stimuli = importer.get_extra_stimuli(extra_stimulus_channel = extra_stimuli_channel, regular_el_stimuli = el_stimuli)
mech_stimuli = importer.get_mechanical_stimuli(force_channel = force_channel, threshold = force_thresh, max_gap_time = 0.005)
actpots = importer.get_action_potentials(max_gap_time = max_gap_time, ap_marker_channels = ap_marker_channels, \
                                       el_stimuli = el_stimuli, mech_stimuli = mech_stimuli)
raw_signal = importer.get_raw_signal_split_by_stimuli(el_stimuli = el_stimuli, verbose = False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


FileNotFoundError: [Errno 2] File b'../data/11_10_27U2a_2000s_2300s_20000Hz_Force_AllFibres.csv' does not exist: b'../data/11_10_27U2a_2000s_2300s_20000Hz_Force_AllFibres.csv'